# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [9]:
%%bash

#!/bin/bash
sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease


Hit:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:4 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:5 https://dl.yarnpkg.com/debian stable InRelease
Hit:6 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Reading package lists...
Building dependency tree...
Reading state information...
32 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
snmp-mibs-downloader is already the newest version (1.2).
snmp is already the newest version (5.8+dfsg-2ubuntu2.9).
snmpd is already the newest version (5.8+dfsg-2ubuntu2.9).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [10]:
%%bash

#!/bin/bash

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.1234.1 /usr/bin/python3 /workspaces/AdvancedNetworks/agent.py
EOF

sudo service snmpd restart
sudo service snmpd status

 * snmpd is running


## Agent code

In [11]:
#!/usr/bin/env python3
import sys
import json

# Path to the JSON file with the SmartWatch data
json_file_path = './smartwatch_data.json'

def load_data():
    try:
        with open(json_file_path, 'r') as file:
            return json.load(file)
    except Exception as e:
        print("Failed to load JSON data:", e)
        sys.exit(1)

# Load the SmartWatch data at the start of the script
smartwatch_data = load_data()



def save_data():
    try:
        with open(json_file_path, 'w') as file:
            json.dump(smartwatch_data, file, indent=4)
        return "Data saved successfully"
    except Exception as e:
        return f"Failed to save JSON data: {e}"

def get_oid_value(oid):
    mappings = {
        ".1.3.6.1.3.1234.1.1.0": ("string", smartwatch_data["status"]),
        ".1.3.6.1.3.1234.1.2.0": ("string", smartwatch_data["name"]),
        ".1.3.6.1.3.1234.1.3.0": ("integer", smartwatch_data["daily_steps"]),
        ".1.3.6.1.3.1234.1.4.0": ("integer", smartwatch_data["battery_level"]),
        ".1.3.6.1.3.1234.1.5.0": ("integer", smartwatch_data["heart_rate"]),
        ".1.3.6.1.3.1234.1.6.0": ("integer", int(smartwatch_data["power_saving_mode"])),
        ".1.3.6.1.3.1234.1.7.0": ("integer", smartwatch_data["calories_burned"]),
        ".1.3.6.1.3.1234.1.8.0": ("integer", smartwatch_data["distance_traveled"]),
        ".1.3.6.1.3.1234.1.9.0": ("integer", int(smartwatch_data["connected_to_smartphone"])),
        ".1.3.6.1.3.1234.1.10.0": ("integer", smartwatch_data["notifications"]),
        ".1.3.6.1.3.1234.1.11.0": ("integer", smartwatch_data["step_goal"]),
        ".1.3.6.1.3.1234.1.12.0": ("integer", int(smartwatch_data["is_charging"])),
        ".1.3.6.1.3.1234.1.13.1": ("integer", smartwatch_data["sleep_cycle"]["deep"]),
        ".1.3.6.1.3.1234.1.13.2": ("integer", smartwatch_data["sleep_cycle"]["light"]),
        ".1.3.6.1.3.1234.1.13.3": ("integer", smartwatch_data["sleep_cycle"]["rem"])
    }
    return mappings if oid is None else mappings.get(oid, (None, None))


def handle_get(oid):
    result = get_oid_value(oid)
    if result[0] is not None:
        return f"{oid}\n{result[0]}\n{result[1]}"
    return "NONE"

def handle_getnext(oid):
    oids = sorted(get_oid_value(None).keys())
    next_oid_index = oids.index(oid) + 1 if oid in oids else 0
    if next_oid_index < len(oids):
        return handle_get(oids[next_oid_index])
    return "NONE"

def handle_set(oid, type, value):
    oid_key = oid.split('.')[-1]
    writable_oids = {
        ".1.3.6.1.3.1234.1.2.0": "name",  # name (read-write)
        ".1.3.6.1.3.1234.1.6.0": "power_saving_mode",  # power_saving_mode (read-write, boolean)
        ".1.3.6.1.3.1234.1.11.0": "step_goal",  # step_goal (read-write)
    }
    if oid in writable_oids:
        if type == "integer":
            smartwatch_data[writable_oids[oid]] = int(value)
        elif type == "string":
            smartwatch_data[writable_oids[oid]] = value
        else:
            return "Unsupported type for SET"
        save_data()
        return f"SET SUCCESS: {oid} set to {value}"
    return "SET FAILURE: OID not writable or does not exist"

def main():
    if len(sys.argv) < 3:
        print("Usage: agent.py -g|-n|-s OID [type] [new_value]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        print(handle_get(oid))
    elif request_type == "-n":  # GETNEXT request
        print(handle_getnext(oid))
    elif request_type == "-s" and len(sys.argv) == 5:  # SET request
        type = sys.argv[3]
        new_value = sys.argv[4]
        print(handle_set(oid, type, new_value))
    else:
        print("NONE")

if __name__ == "__main__":
    main()


Usage: agent.py -g|-n|-s OID [type] [new_value]


## MIB

In [12]:
%%writefile MYMIB.txt

SMARTWATCH-MIB DEFINITIONS ::= BEGIN

IMPORTS
    MODULE-IDENTITY, OBJECT-TYPE, Integer32
        FROM SNMPv2-SMI
    DisplayString, TruthValue
        FROM SNMPv2-TC;

smartWatchMIB MODULE-IDENTITY
    LAST-UPDATED "202407080000Z"
    ORGANIZATION "PUCRS"
    CONTACT-INFO
        "admin@localhost"
    DESCRIPTION
        "MIB for accessing information about a smartwatch"
    ::= { experimental 54321 }

smartWatch OBJECT IDENTIFIER ::= { smartWatchMIB 1 }

status OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Current state of the smart watch (on or off)"
    ::= { smartWatch 1 }

name OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION
        "The smart watch name"
    ::= { smartWatch 2 }

dailySteps OBJECT-TYPE
    SYNTAX Integer32
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Total number of steps in the day"
    ::= { smartWatch 3 }

batteryLevel OBJECT-TYPE
    SYNTAX Integer32 (0..100)
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Battery percentage of the smart watch"
    ::= { smartWatch 4 }

heartRate OBJECT-TYPE
    SYNTAX Integer32
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Heart rate detected by the smart watch"
    ::= { smartWatch 5 }

powerSavingMode OBJECT-TYPE
    SYNTAX TruthValue
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION
        "Indicates if power saving mode is on or off"
    ::= { smartWatch 6 }

caloriesBurned OBJECT-TYPE
    SYNTAX Integer32
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Burned calories during the day"
    ::= { smartWatch 7 }

distanceTraveled OBJECT-TYPE
    SYNTAX Integer32
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Distance travelled in a day"
    ::= { smartWatch 8 }

connectedToSmartphone OBJECT-TYPE
    SYNTAX TruthValue
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Indicates if the smart watch is connected to a phone"
    ::= { smartWatch 9 }

notifications OBJECT-TYPE
    SYNTAX Integer32
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Number of notifications received on the smart watch"
    ::= { smartWatch 10 }

stepGoal OBJECT-TYPE
    SYNTAX Integer32
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION
        "Steps goal defined for the day"
    ::= { smartWatch 11 }

isCharging OBJECT-TYPE
    SYNTAX TruthValue
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Indicates if the smart watch is being charged"
    ::= { smartWatch 12 }

sleepCycleTable OBJECT-TYPE
    SYNTAX SEQUENCE OF sleepPhase
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Sleep phases and how many minutes the user got of each one"
    ::= { smartWatch 13 }

sleepPhase OBJECT-TYPE
    SYNTAX SleepPhase
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Sleep phase"
    INDEX { sleepPhaseId }
    ::= { sleepCycleTable 1 }

SleepPhase ::= SEQUENCE {
    sleepPhaseName DisplayString,
    sleepingTime Integer32
}

sleepPhaseId OBJECT-TYPE
    SYNTAX Integer32 { deep_sleep(1), light_sleep(2), rem(3)}
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Sleep phase ID"
    ::= { sleepPhase 1 }

sleepPhaseName OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Sleep phase name"
    ::= { sleepPhase 2 }

sleepingTime OBJECT-TYPE
    SYNTAX Integer32
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Time spent sleeping"
    ::= { sleepPhase 3 }

END


Overwriting MYMIB.txt


## Consultas

In [17]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.1.0 
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.2.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.3.0 
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.4.0 
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.5.0 
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.6.0 
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.7.0 
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.8.0 
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.9.0 
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.10.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.11.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.12.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.13.1
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.13.2
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.13.3

snmpgetnext -v2c -c public localhost .1.3.6.1.3.1234.1.1
snmpgetnext -v2c -c public localhost .1.3.6.1.3.1234.1.1.1.1
snmpgetnext -v2c -c public localhost .1.3.6.1.3.1234.1.1.1.1.1
snmpgetnext -v2c -c public localhost .1.3.6.1.3.1234.1.1.1.1.2
snmpgetnext -v2c -c public localhost .1.3.6.1.3.1234.1.1.1.1.3
snmpgetnext -v2c -c public localhost .1.3.6.1.3.1234.1.1.1.2.1
snmpgetnext -v2c -c public localhost .1.3.6.1.3.1234.1.1.1.2.2

SNMPv2-SMI::experimental.1234.1.1.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.2.0 = No Such Instance currently exists at this OID


SNMPv2-SMI::experimental.1234.1.3.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.4.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.5.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.6.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.7.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.8.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.9.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.10.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.11.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.12.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.13.1 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.13.2 = No Such Instance currently exists

## Set

In [14]:
%%bash

snmpset -v2c -c private localhost .1.3.6.1.3.1234.1.1.0 s "off"
snmpset -v2c -c private localhost .1.3.6.1.3.1234.1.11.0 integer 15000

SNMPv2-SMI::experimental.1234.1.1.0 = STRING: "off"
SNMPv2-SMI::experimental.1234.1.11.0 = INTEGER: 15000


## Consulta atualizada

In [16]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.1.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.11.0

SNMPv2-SMI::experimental.1234.1.1.0 = No Such Instance currently exists at this OID
SNMPv2-SMI::experimental.1234.1.11.0 = No Such Instance currently exists at this OID
